In [ ]:
'''Permanent link
https://huggingface.co/spaces/nnsohamnn/foodmodel'''

In [ ]:
#Temporary Deployment Link https://colab.research.google.com/drive/1zb_FGQpzVfyxgQycU9CTfnDRdhVSQyAw?authuser=1#scrollTo=usFEWUYmrBUQ

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
import urllib.request
import zipfile
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
import requests
from pathlib import Path

# Download helper functions from Learn PyTorch repo
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download...")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)


helper_functions.py already exists, skipping download...


In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
import os
import shutil

In [ ]:
selected_classes = ['chicken_curry', 'fried_rice', 'ice_cream', 'pizza', 'samosa']

In [ ]:
root_dir = './data'

food101_dataset = datasets.Food101(root=root_dir, download=True)

100%|██████████| 4996278331/4996278331 [03:31<00:00, 23606722.81it/s]


Extracting ./data/food-101.tar.gz to ./data


In [ ]:
selected_data_dir = './data/selected_classes'
os.makedirs(selected_data_dir, exist_ok=True)


food101_path = os.path.join(root_dir, 'food-101', 'images')

for class_name in selected_classes:
    src_dir = os.path.join(food101_path, class_name)
    dst_dir = os.path.join(selected_data_dir, class_name)
    shutil.copytree(src_dir, dst_dir)

print(f'Selected classes have been copied to {selected_data_dir}')

Selected classes have been copied to ./data/selected_classes


In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


selected_dataset = datasets.ImageFolder(root=selected_data_dir, transform=transform)


train_size = int(0.8 * len(selected_dataset))
test_size = len(selected_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(selected_dataset, [train_size, test_size])


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'Training and validation data loaders are ready.')


Training and validation data loaders are ready.


In [ ]:
model = torchvision.models.mobilenet_v2(pretrained=True)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 123MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)
from helper_functions import accuracy_fn

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
epochs = 10

for epoch in range(epochs):
    print(f"Epoch: {epoch+1}\n-----")
    train_loss = 0
    model.train()

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)


        y_pred = model(X)
        loss = loss_fn(y_pred, y)


        train_loss += loss.item()


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)

    test_loss, test_acc = 0, 0
    model.eval()

    with torch.inference_mode():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)


            test_pred = model(X_test)


            test_loss += loss_fn(test_pred, y_test).item()
            test_acc += accuracy_fn(y_test, test_pred.argmax(dim=1))


    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    print(f"Train loss: {train_loss:.5f} | Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}")


Epoch: 1
-----
Train loss: 0.93013 | Test loss: 0.42596 | Test acc: 85.45
Epoch: 2
-----
Train loss: 0.35401 | Test loss: 0.34320 | Test acc: 88.48
Epoch: 3
-----
Train loss: 0.29105 | Test loss: 0.34922 | Test acc: 89.06
Epoch: 4
-----
Train loss: 0.24530 | Test loss: 0.45604 | Test acc: 84.96
Epoch: 5
-----
Train loss: 0.21882 | Test loss: 0.28059 | Test acc: 91.02
Epoch: 6
-----
Train loss: 0.17892 | Test loss: 0.53517 | Test acc: 84.18
Epoch: 7
-----
Train loss: 0.19588 | Test loss: 0.43246 | Test acc: 86.33
Epoch: 8
-----
Train loss: 0.15977 | Test loss: 0.56223 | Test acc: 83.50
Epoch: 9
-----
Train loss: 0.15825 | Test loss: 0.45101 | Test acc: 87.79
Epoch: 10
-----
Train loss: 0.11396 | Test loss: 0.55437 | Test acc: 85.74


In [ ]:

torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
from PIL import Image

In [ ]:
transform1 = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

In [ ]:
def predict(image_path):
    image = Image.open(image_path)
    input_tensor = transform(image).unsqueeze(0)
    input_tensor = input_tensor.to('cuda')

    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)


    predicted_label = selected_classes[predicted.item()]
    return predicted_label

In [ ]:
predict('/content/istockphoto-502663991-612x612.jpg')

'samosa'

In [ ]:
model=torchvision.models.mobilenet_v2(pretrained=False)

state_dict = torch.load('model_weights.pth')
model.load_state_dict(state_dict)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [ ]:
def predict_with_probabilities(model, image_path, class_names):

  model.eval()

  transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])

  with torch.no_grad():
      image = Image.open(image_path)
      input_tensor = transform(image).unsqueeze(0)
      input_tensor = input_tensor.to('cuda')

      output = model(input_tensor)
      probabilities = torch.softmax(output, dim=1)
      probabilities = probabilities.cpu().numpy()[0]


      print("Number of probabilities from model:", len(probabilities))
      print("Number of classes provided:", len(class_names))

  class_probabilities = {}

  for i in range(min(len(probabilities), len(class_names))):
      class_probabilities[class_names[i]] = probabilities[i]

  return class_probabilities

In [ ]:
model=torchvision.models.mobilenet_v2(pretrained=False)

state_dict = torch.load('model_weights.pth')
model.load_state_dict(state_dict)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
#predict_with_probabilities(model, '/content/istockphoto-502663991-612x612.jpg', selected_classes)


Number of probabilities from model: 1000
Number of classes provided: 5


{'chicken_curry': 0.004912394,
 'fried_rice': 0.0011723645,
 'ice_cream': 0.011019544,
 'pizza': 3.460816e-05,
 'samosa': 0.98199505}